# Tβ1-10: Speed Dependence (10 epochs/λ)

**Purpose**: Complete speed dependence comparison with slowest sweep

**Protocol**:
- Ordered branch: λ 0.30→0.70, 10 ep/λ
- Collapse branch: λ 0.70→0.30, 10 ep/λ
- Compare with Tα1/Tα2 (1, 3 ep/λ) results

**Expected**: Gap remains even with slow sweep

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, glob, json, time
from datetime import datetime

EXP_NAME = 'exp_Tb1_speed_10ep'
NOTEBOOK_ID = 'Tb1-10'
BASE_DIR = '/content/drive/MyDrive/dual-gradient-learning/Paper-A'

# Checkpoint directories
ORDERED_CKPT_DIR = sorted(glob.glob(f'{BASE_DIR}/exp_Ta_prep_ordered_*'))[-1] + '/checkpoints'
COLLAPSE_CKPT_DIR = sorted(glob.glob(f'{BASE_DIR}/exp_Ta_prep_collapse_*'))[-1] + '/checkpoints'
print(f'Ordered checkpoints: {ORDERED_CKPT_DIR}')
print(f'Collapse checkpoints: {COLLAPSE_CKPT_DIR}')

existing = glob.glob(f'{BASE_DIR}/{EXP_NAME}_*')
if existing:
    SAVE_DIR = sorted(existing)[-1]
    print(f'🔄 Resuming: {SAVE_DIR}')
else:
    TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
    SAVE_DIR = f'{BASE_DIR}/{EXP_NAME}_{TIMESTAMP}'
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f'🆕 New: {SAVE_DIR}')

os.makedirs(f'{SAVE_DIR}/figures', exist_ok=True)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name()}')

In [ ]:
# Experiment parameters
BATCH_SIZE = 256
NUM_WORKERS = 4
LR = 0.1
K = 16

NOISE_RATE = 0.4
LAMBDA_START_UP = 0.30
LAMBDA_END_UP = 0.70
LAMBDA_START_DOWN = 0.70
LAMBDA_END_DOWN = 0.30
LAMBDA_STEP = 0.02
EPOCHS_PER_LAMBDA = 10  # Slow sweep

LAMBDA_GRID_UP = np.arange(LAMBDA_START_UP, LAMBDA_END_UP + LAMBDA_STEP/2, LAMBDA_STEP)
LAMBDA_GRID_UP = np.round(LAMBDA_GRID_UP, 2)
LAMBDA_GRID_DOWN = np.arange(LAMBDA_START_DOWN, LAMBDA_END_DOWN - LAMBDA_STEP/2, -LAMBDA_STEP)
LAMBDA_GRID_DOWN = np.round(LAMBDA_GRID_DOWN, 2)

N_SEEDS = 5

print(f'η = {NOISE_RATE}')
print(f'Epochs per λ: {EPOCHS_PER_LAMBDA}')
print(f'λ points: {len(LAMBDA_GRID_UP)}')
print(f'Total epochs per run: {len(LAMBDA_GRID_UP) * EPOCHS_PER_LAMBDA}')

In [ ]:
def get_resnet18():
    model = resnet18(weights=None, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    return model

class IndexedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def inject_label_noise(labels, noise_rate, seed):
    np.random.seed(seed)
    noisy = labels.copy()
    n_noisy = int(noise_rate * len(labels))
    idx = np.random.choice(len(labels), n_noisy, replace=False)
    for i in idx:
        noisy[i] = np.random.choice([l for l in range(10) if l != labels[i]])
    return noisy

def load_cifar10():
    tr = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(),
                             transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))])
    te = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))])
    return torchvision.datasets.CIFAR10('./data', True, tr, download=True), torchvision.datasets.CIFAR10('./data', False, te, download=True)

def evaluate(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            correct += (model(x).argmax(1) == y).sum().item()
            total += y.size(0)
    return correct / total

In [ ]:
def train_one_epoch(model, train_loader, opt, clean_t, noisy_t, lam, cached_gv_ref):
    crit = nn.CrossEntropyLoss()
    model.train()
    step = cached_gv_ref['step']
    cached_gv = cached_gv_ref['gv']
    
    for x, _, idx in train_loader:
        x, idx = x.to(device), idx.to(device)
        bn, bc = noisy_t[idx], clean_t[idx]
        
        opt.zero_grad()
        loss_s = crit(model(x), bn)
        loss_s.backward(retain_graph=True)
        gs = parameters_to_vector([p.grad for p in model.parameters()]).clone()
        
        if step % K == 0 or cached_gv is None:
            opt.zero_grad()
            loss_v = crit(model(x), bc)
            loss_v.backward()
            cached_gv = parameters_to_vector([p.grad for p in model.parameters()]).clone()
        
        gs_n = gs / (gs.norm() + 1e-12)
        gv_n = cached_gv / (cached_gv.norm() + 1e-12)
        
        g_mix = (1 - lam) * gs_n + lam * gv_n
        opt.zero_grad()
        i = 0
        for p in model.parameters():
            n = p.numel()
            p.grad = g_mix[i:i+n].view(p.shape).clone()
            i += n
        opt.step()
        step += 1
    
    cached_gv_ref['step'] = step
    cached_gv_ref['gv'] = cached_gv

In [ ]:
def run_sweep(ckpt_path, train_loader, test_loader, clean_labels, noisy_labels, direction='up'):
    """Run sweep from checkpoint"""
    ckpt = torch.load(ckpt_path, map_location=device)
    seed = ckpt['seed']
    init_error = ckpt['final_error']
    
    lambda_grid = LAMBDA_GRID_UP if direction == 'up' else LAMBDA_GRID_DOWN
    branch = 'ordered_up' if direction == 'up' else 'collapse_down'
    
    print(f'    Loaded: seed={seed}, init_error={init_error:.4f}, direction={direction}')
    
    clean_t = torch.tensor(clean_labels, device=device)
    noisy_t = torch.tensor(noisy_labels, device=device)
    
    set_seed(seed + 1000)
    model = get_resnet18().to(device)
    model.load_state_dict(ckpt['model_state_dict'])
    
    opt = optim.SGD(model.parameters(), lr=LR * 0.01, momentum=0.9, weight_decay=5e-4)
    cached_gv_ref = {'step': 0, 'gv': None}
    trajectory = []
    
    print(f'    Sweeping ({EPOCHS_PER_LAMBDA} ep/λ)...')
    
    for lam in lambda_grid:
        for _ in range(EPOCHS_PER_LAMBDA):
            train_one_epoch(model, train_loader, opt, clean_t, noisy_t, lam, cached_gv_ref)
        
        err = 1 - evaluate(model, test_loader)
        trajectory.append({'lambda': float(lam), 'error': err})
        print(f'      λ={lam:.2f}: err={err:.4f}')
    
    return {
        'seed': seed,
        'init_error': init_error,
        'final_error': trajectory[-1]['error'],
        'branch': branch,
        'epochs_per_lambda': EPOCHS_PER_LAMBDA,
        'trajectory': trajectory,
        'checkpoint_source': os.path.basename(ckpt_path)
    }

In [ ]:
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader = DataLoader(IndexedDataset(trainset), BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(testset, BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

# Get checkpoint files
ordered_ckpts = sorted(glob.glob(f'{ORDERED_CKPT_DIR}/ordered_seed*.pth'))[:N_SEEDS]
collapse_ckpts = sorted(glob.glob(f'{COLLAPSE_CKPT_DIR}/collapse_seed*.pth'))[:N_SEEDS]

# Filter out 90% collapse checkpoints
collapse_ckpts_filtered = []
for cp in collapse_ckpts:
    ckpt = torch.load(cp, map_location='cpu')
    if ckpt['final_error'] < 0.85:  # Exclude 90% trapped
        collapse_ckpts_filtered.append(cp)
collapse_ckpts = collapse_ckpts_filtered[:N_SEEDS]

print(f'Ordered checkpoints: {len(ordered_ckpts)}')
print(f'Collapse checkpoints (non-90%): {len(collapse_ckpts)}')

m = get_resnet18().to(device)
for _ in range(10): _ = m(torch.randn(BATCH_SIZE,3,32,32,device=device))
del m; torch.cuda.empty_cache()
print('Ready')

In [ ]:
results = []
ckpt_file = f'{SAVE_DIR}/{NOTEBOOK_ID}_checkpoint.json'
done_keys = set()

if os.path.exists(ckpt_file):
    results = json.load(open(ckpt_file))
    done_keys = {(r['checkpoint_source'], r['branch']) for r in results}
    print(f'Loaded: {len(done_keys)} done')

# Run ordered branch (up)
print('\n' + '='*60)
print('ORDERED BRANCH (λ ↑)')
print('='*60)

for i, ckpt_path in enumerate(ordered_ckpts):
    ckpt_basename = os.path.basename(ckpt_path)
    if (ckpt_basename, 'ordered_up') in done_keys:
        continue
    
    print(f'\n[{i+1}/{len(ordered_ckpts)}] {ckpt_basename}')
    
    ckpt_temp = torch.load(ckpt_path, map_location='cpu')
    seed = ckpt_temp['seed']
    noisy_labels = inject_label_noise(clean_labels, NOISE_RATE, seed)
    
    t0 = time.time()
    result = run_sweep(ckpt_path, train_loader, test_loader, clean_labels, noisy_labels, 'up')
    result['time'] = time.time() - t0
    result['experiment_id'] = f'{NOTEBOOK_ID}-ord-{i+1:03d}'
    
    results.append(result)
    done_keys.add((ckpt_basename, 'ordered_up'))
    json.dump(results, open(ckpt_file, 'w'), indent=2, default=str)
    
    print(f'  Init: {result["init_error"]:.2%} → Final: {result["final_error"]:.2%} | {result["time"]/60:.1f}min')
    torch.cuda.empty_cache()

# Run collapse branch (down)
print('\n' + '='*60)
print('COLLAPSE BRANCH (λ ↓)')
print('='*60)

for i, ckpt_path in enumerate(collapse_ckpts):
    ckpt_basename = os.path.basename(ckpt_path)
    if (ckpt_basename, 'collapse_down') in done_keys:
        continue
    
    print(f'\n[{i+1}/{len(collapse_ckpts)}] {ckpt_basename}')
    
    ckpt_temp = torch.load(ckpt_path, map_location='cpu')
    seed = ckpt_temp['seed']
    noisy_labels = inject_label_noise(clean_labels, NOISE_RATE, seed)
    
    t0 = time.time()
    result = run_sweep(ckpt_path, train_loader, test_loader, clean_labels, noisy_labels, 'down')
    result['time'] = time.time() - t0
    result['experiment_id'] = f'{NOTEBOOK_ID}-col-{i+1:03d}'
    
    results.append(result)
    done_keys.add((ckpt_basename, 'collapse_down'))
    json.dump(results, open(ckpt_file, 'w'), indent=2, default=str)
    
    print(f'  Init: {result["init_error"]:.2%} → Final: {result["final_error"]:.2%} | {result["time"]/60:.1f}min')
    torch.cuda.empty_cache()

print('\n' + '='*60)
print(f'{NOTEBOOK_ID} COMPLETE')
print('='*60)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

json.dump(results, open(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.json', 'w'), indent=2, default=str)

# Separate branches
ordered_results = [r for r in results if r['branch'] == 'ordered_up']
collapse_results = [r for r in results if r['branch'] == 'collapse_down']

# Create DataFrames
all_data = []
for r in results:
    for t in r['trajectory']:
        all_data.append({'seed': r['seed'], 'lambda': t['lambda'], 'error': t['error'], 'branch': r['branch']})
df = pd.DataFrame(all_data)
df.to_csv(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.csv', index=False)

print(f'Ordered runs: {len(ordered_results)}')
print(f'Collapse runs: {len(collapse_results)}')

In [ ]:
# Visualization
fig, ax = plt.subplots(figsize=(10, 6))

# Plot ordered branch
df_ord = df[df['branch'] == 'ordered_up']
if len(df_ord) > 0:
    df_ord_mean = df_ord.groupby('lambda')['error'].agg(['mean', 'std']).reset_index()
    ax.fill_between(df_ord_mean['lambda'], df_ord_mean['mean'] - df_ord_mean['std'], 
                    df_ord_mean['mean'] + df_ord_mean['std'], alpha=0.3, color='blue')
    ax.plot(df_ord_mean['lambda'], df_ord_mean['mean'], 'b-o', linewidth=2, markersize=5, label='Ordered (λ↑)')

# Plot collapse branch
df_col = df[df['branch'] == 'collapse_down']
if len(df_col) > 0:
    df_col_mean = df_col.groupby('lambda')['error'].agg(['mean', 'std']).reset_index()
    ax.fill_between(df_col_mean['lambda'], df_col_mean['mean'] - df_col_mean['std'],
                    df_col_mean['mean'] + df_col_mean['std'], alpha=0.3, color='red')
    ax.plot(df_col_mean['lambda'], df_col_mean['mean'], 'r-s', linewidth=2, markersize=5, label='Collapse (λ↓)')

ax.axhline(0.40, color='orange', linestyle='--', linewidth=1.5, alpha=0.7, label='Collapse threshold')
ax.axhline(0.20, color='green', linestyle='--', linewidth=1.5, alpha=0.7, label='Ordered threshold')

ax.set_xlabel('λ', fontsize=12)
ax.set_ylabel('Test Error', fontsize=12)
ax.set_title(f'Hysteresis Loop ({EPOCHS_PER_LAMBDA} epochs/λ) - Speed Dependence', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_xlim(0.28, 0.72)

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/figures/{NOTEBOOK_ID}_hysteresis.png', dpi=150)
plt.show()

# Summary
if len(df_ord) > 0 and len(df_col) > 0:
    gap_050 = df_col_mean[df_col_mean['lambda']==0.50]['mean'].values[0] - df_ord_mean[df_ord_mean['lambda']==0.50]['mean'].values[0]
    print(f'\n📊 Gap at λ=0.50: {gap_050*100:.1f}%')
    print(f'   Ordered @ λ=0.50: {df_ord_mean[df_ord_mean["lambda"]==0.50]["mean"].values[0]*100:.1f}%')
    print(f'   Collapse @ λ=0.50: {df_col_mean[df_col_mean["lambda"]==0.50]["mean"].values[0]*100:.1f}%')